In [25]:
import tensorflow as tf
import numpy as np

EPOCHS = 10

#DenseUnit 구현

In [26]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit,self).__init__()
        #free activation구조는 하고
        #concatenation구조를 해야함
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size, padding = 'same')
        self.concat = tf.keras.layers.Concatenate()

    def __call__(self, x , training = False, mask = None):
        #x : BATCH , H, W, Channel_in)
        #batchnom -> relu -> conv 는 free activation구조다.
        h = self.bn(x, training = training)
        h = tf.nn.relu(h)
        h = self.conv(h)#h : (Batch, H, W, filter_out)
        #들어온 입력과 출력을 concat한다.
        return self.concat([x,h]) #(Batch, H, W , (ch_in + filter_out))

#DenseLayer 구현

In [27]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit,growth_rate, kernel_size):
        super(DenseLayer,self).__init__()
        self.sequence = list()
        for _ in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))#growth_rate만큼 붙여주는게 Densenet

    def __call__(self, x, training = False, mask = None):
        for unit in self.sequence:
            x = unit(x,training = training)
        return x

#TransitionLayer 구현

In [28]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        #denselayer에서 growth_rate가 num_unit에 비해 너무 많으면 안됨 그래서 transition을 통해 채널개수를 줄여줌
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding = 'same')
        self.pool = tf.keras.layers.MaxPool2D((2,2))

    def __call__(self, x, training = False, make = None):
        x = self.conv(x)
        return self.pool(x)

#모델 정의


In [29]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8,(3,3), padding='same',activation = 'relu')

        self.dl1 = DenseLayer(2, 4, (3,3))#28x28x16 unit2개 growthrate 4니까 2*2*4
        self.tr1 = TransitionLayer(16,(3,3))#14x14x16

        self.dl2 = DenseLayer(2, 8, (3,3))#14x14x32
        self.tr2 = TransitionLayer(32,(3,3))#7x7x32

        self.dl3 = DenseLayer(2, 16, (3,3))#7x7x64

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128,activation='relu')
        self.dense2 = tf.keras.layers.Dense(10,activation='softmax')
    def __call__(self,x,training = False, mask = None):
        x = self.conv1(x)

        x = self.dl1(x, training = training)
        x = self.tr1(x)

        x = self.dl2(x, training = training)
        x = self.tr2(x)

        x = self.dl3(x, training = training)

        x = self.flatten(x)
        x = self.dense1(x)

        return self.dense2(x)

#학습 루프 정의

In [30]:
@tf.function
def train_step(model,images,labels,loss_object,optimizer,train_loss,train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(images,training = True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)
  
@tf.function
def test_step(model,images,labels,loss_object,test_loss,test_accuracy):
  predictions = model(images, training = False)

  t_loss = loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)

#데이터셋

In [31]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0 #이렇게하면 float64로 됨

x_train = x_train[...,tf.newaxis].astype(np.float32)
x_test = x_test[...,tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)


#학습환경정의

In [32]:
model = DenseNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

#학습

In [33]:
for epoch in range(EPOCHS):
  for images,labels in train_ds:
    train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_images, test_labels in test_ds:
    test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

  template = 'Epoch:{}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch:1, Loss: 0.1170617863535881, Accuracy: 96.55833435058594, Test Loss: 0.061431970447301865, Test Accuracy: 98.25
Epoch:2, Loss: 0.05897001177072525, Accuracy: 98.30667114257812, Test Loss: 0.0652400329709053, Test Accuracy: 97.98999786376953
Epoch:3, Loss: 0.04468084126710892, Accuracy: 98.73832702636719, Test Loss: 0.04304403066635132, Test Accuracy: 98.77999877929688
Epoch:4, Loss: 0.04006343334913254, Accuracy: 98.8566665649414, Test Loss: 0.07941723614931107, Test Accuracy: 97.97999572753906
Epoch:5, Loss: 0.038685183972120285, Accuracy: 98.93000030517578, Test Loss: 0.04936480149626732, Test Accuracy: 98.87999725341797
Epoch:6, Loss: 0.03271959722042084, Accuracy: 99.10166931152344, Test Loss: 0.057595133781433105, Test Accuracy: 98.68999481201172
Epoch:7, Loss: 0.031006233766674995, Accuracy: 99.211669921875, Test Loss: 0.07564510405063629, Test Accuracy: 98.61000061035156
Epoch:8, Loss: 0.025647856295108795, Accuracy: 99.31666564941406, Test Loss: 0.06407444924116135, Test 